In [ ]:
import dagshub
import mlflow
import sys
import torch
import json
import numpy as np
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint


In [ ]:
path = Path.cwd().parent /"src"
sys.path.append(str(path))

In [ ]:
dagshub.init(
    repo_owner="stephenjera",
    repo_name="Genre-Classification",
    mlflow=True,
)

In [ ]:
mlflow.set_tracking_uri('http://localhost:5000')

In [ ]:
model_uri = "models:/genre-classifier/5"
loaded_model = mlflow.pytorch.load_model(model_uri)
#loaded_model = mlflow.pyfunc.load_model(model_uri)

In [ ]:
def load_data(dataset_path):
    """
    Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(dataset_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    # X = np.array(data["spectrogram"])
    y = np.array(data["labels"])
    mappings = data["mappings"]
    return X, y, mappings

In [ ]:
dataset_path = Path.cwd().parent / "data" / "processed" / "genres_mfccs.json"

In [ ]:
# Load one sample from data 
X, y, mappings = load_data(dataset_path)

In [ ]:
X = torch.tensor(X, dtype=torch.float32)

In [ ]:
#X = np.array(X, dtype=np.float128)
#X = torch.tensor(X, dtype=torch.int64).clone().detach()
single_mfcc = X[:1].tolist()
#single_mfcc = single_mfcc.astype(np.float64).tolist()
#single_mfcc = torch.from_numpy(X[:1]).float().tolist()

# Make prediction request
url = 'http://localhost:1234/invocations'
headers = {'Content-Type': 'application/json'}
data = {
  "instances": single_mfcc
}
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print prediction
pprint(response.json())


In [ ]:
predictions = response.json()['predictions']

# Convert the predictions to a numpy array
predictions_array = np.array(predictions)

# Find the index of the maximum value
argmax_index = np.argmax(predictions_array)

print(argmax_index)

In [ ]:
prediction = loaded_model.predict_step(torch.tensor(X[:2]))

In [ ]:
prediction.argmax()

In [ ]:
# Get the predicted class
predicted_class_index = 9#prediction.argmax().item()

print(f"The predicted class index is {predicted_class_index}")

In [ ]:
reverse_dict = {v: k for k, v in mappings.items()}

In [ ]:
print(
    f"prediction:{predicted_class_index}, {reverse_dict[predicted_class_index]} Actual {y[1]}, {reverse_dict[y[1]]}"
)